In [1]:
import re
import os
import pickle
import numpy as np
import pandas as pd
import json
import logging
import torch
import math
import time
from torch.utils.data import Dataset
from utils.utils import newsample, getId2idx, tokenize, getVocab, my_collate, Partition_Sampler
from data.configs.demo import config
from torch.utils.data import DataLoader
from collections import defaultdict
from transformers import BertTokenizer,BertModel,BertTokenizerFast,DebertaTokenizer,DebertaTokenizerFast, AutoTokenizer
from utils.MIND import MIND
from utils.Manager import Manager

logger = logging.getLogger(__name__)

In [2]:
# t = AutoTokenizer.from_pretrained('bert-base-uncased', cache_dir=config.path + "bert_cache/")
# t2 = DebertaTokenizerFast.from_pretrained('microsoft/deberta-base', cache_dir=config.path + "bert_cache/")

In [4]:
# config.reducer = 'bm25'
# config.reducer = 'bow'
# config.reducer = 'matching'

# config.signal_length = 10
# config.scale = 'large'
# config.impr_size = 100
# config.mode = 'test'

# config.bert = 'microsoft/deberta-base'
# config.embedding = 'deberta'

# config.bert = 'bert-base-uncased
# config.embedding = 'bert'

manager = Manager(config)
path = manager.path + 'MIND/MINDdemo_train/'
a = MIND(manager, path + 'news.tsv', path + 'behaviors.tsv')

[2021-09-23 11:03:31,267] INFO (utils.MIND) process NO.0 loading cached user behavior from data/cache/bert/MINDdemo_train/10/behaviors..pkl
[2021-09-23 11:03:31,287] INFO (utils.MIND) process NO.0 loading cached news tokenization from data/cache/bert/MINDdemo_train/news.pkl
[2021-09-23 11:03:32,179] INFO (utils.utils) deduplicating...


In [5]:
x = a[1]
x

{'user_index': array([2]),
 'cdd_id': array([32187, 41306, 37405, 44370, 45366]),
 'his_id': array([29991,  1537, 28125,  6875, 20529, 25906, 26638,  1082, 13995,
        13919, 27868, 10693, 22638, 25937,  5642, 19188,   129, 10970,
         4853, 22037, 26144,  2600,  4501,  2757,  8178, 14857, 30635,
        17680, 28583, 28646, 17080, 29418, 28192, 25725, 15871, 17164,
        23107, 28920,  2394, 28044, 21717, 22954, 26998, 12733, 14202,
        13463, 12087,  8524,  2334, 25816]),
 'cdd_encoded_index': array([[ 2053,  1012,  1015,  6501,  2194, 18806, 10528, 13463,  4530,
          1999,  5157,  4670,  9440, 11248,  1037,  5109, 10052,  4530,
          1999,  6501,  8381,  2008,  2038,  2464,  2111,  2735,  2000,
         15955,  2066, 14904,  1010, 10869,  1998, 26011,  6501,  1012,
          5446,  1011,  3316,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     

In [ ]:
cdd = x['cdd_encoded_index'][0]

In [ ]:
subword = x['cdd_subword_index'][0]

In [ ]:
manager.convert_tokens_to_words(t.convert_ids_to_tokens(cdd))

In [ ]:
subword

In [ ]:
t.convert_ids_to_tokens(cdd)

In [ ]:
t.convert_ids_to_tokens([   1, 7565,   16, 4690,    0,    0,    0,    0])

In [ ]:
tokens = t.tokenize("[CLS] I love you embeddings.")
tokens

In [ ]:
news = pickle.load(open('/data/workspace/Peitian/Code/Document-Reduction/Code/data/cache/deberta/MINDdemo_train/news.pkl', 'rb'))
ids = news['encoded_news']
subwords = news['subwords_all']

In [ ]:
t.convert_tokens_to_ids([i for i in r"[.&*()+=/\<>,!?;:~`@#$%^]"])